In [2]:
import struct
import open3d as o3d
import numpy as np
from OpenGL.GL import *

# Define a structure to hold the vertex data
class Vertex:
    def __init__(self, x, y, z, r, g, b, nx, ny, nz):
        self.x = x
        self.y = y
        self.z = z
        self.r = r
        self.g = g
        self.b = b
        self.nx = nx
        self.ny = ny
        self.nz = nz

# Load the PLY file and extract the vertex data
def loadPointCloud(filename):
    with open(filename, 'rb') as f:
        # Read the header information
        header = f.readline().decode().strip()
        if header != 'ply':
            raise ValueError('Not a PLY file')
        while True:
            line = f.readline().decode().strip()
            if line == 'end_header':
                break
            if line.startswith('element vertex'):
                numVertices = int(line.split()[2])
        # Read the vertex data
        vertices = []
        for i in range(numVertices):
            x, y, z, r, g, b, nx, ny, nz = struct.unpack('<fffffffff', f.read(36))
            vertices.append(Vertex(x, y, z, r, g, b, nx, ny, nz))
    return vertices

def main():
    # Load the point cloud data
    vertices = np.array(loadPointCloud("logs/chair_test/chair_surfels_10k_pd_bluens.ply"), dtype=np.float32)

    # Create and bind a Vertex Buffer Object (VBO)
    vbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, vbo)

    # Upload the vertex data to the VBO
    glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

    # Set up the vertex attribute pointers
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(0))
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(12))
    glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(24))
    glEnableVertexAttribArray(0)
    glEnableVertexAttribArray(1)
    glEnableVertexAttribArray(2)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import numpy as np

# Define the size of the window
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

# Define the camera position
CAMERA_POS = (0, 0, 2)

# Define the point cloud color
COLOR = (1, 1, 1)

def main():
    # Initialize Pygame
    pygame.init()
    pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT), DOUBLEBUF|OPENGL)

    # Set up the OpenGL viewport
    glViewport(0, 0, WINDOW_WIDTH, WINDOW_HEIGHT)
    glClearColor(0, 0, 0, 1)
    glEnable(GL_DEPTH_TEST)
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)

    # Set up the projection matrix
    glMatrixMode(GL_PROJECTION)
    gluPerspective(45, WINDOW_WIDTH/WINDOW_HEIGHT, 0.1, 100)

    # Load the point cloud data
    vertices =o3d.io.read_point_cloud(("logs/chair_test/chair_surfels_10k_pd_bluens.ply"), dtype=np.float32)

    # Create and bind a Vertex Buffer Object (VBO)
    vbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, vbo)

    # Upload the vertex data to the VBO
    glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

    # Set up the vertex attribute pointers
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(0))
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(12))
    glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, 36, ctypes.c_void_p(24))
    glEnableVertexAttribArray(0)
    glEnableVertexAttribArray(1)
    glEnableVertexAttribArray(2)

    # Start the main loop
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

        # Clear the screen and depth buffer
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        # Set up the modelview matrix
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        gluLookAt(CAMERA_POS[0], CAMERA_POS[1], CAMERA_POS[2], 0, 0, 0, 0, 1, 0)

        # Draw the point cloud
        glColor3f(*COLOR)
        glPointSize(2)
        glBindBuffer(GL_ARRAY_BUFFER, vbo)
        glDrawArrays(GL_POINTS, 0, len(vertices))

        # Swap the buffers
        pygame.display.flip()

if __name__ == '__main__':
    main()

TypeError: read_point_cloud(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, format: str = 'auto', remove_nan_points: bool = False, remove_infinite_points: bool = False, print_progress: bool = False) -> open3d.cpu.pybind.geometry.PointCloud

Invoked with: 'logs/chair_test/chair_surfels_10k_pd_bluens.ply'; kwargs: dtype=<class 'numpy.float32'>